In [6]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

wd =os.getcwd()
#glacier = "petermann"
#glacier = "jakobshavn"
#glacier ="kronebreen"
#glacier ="helheim"
#glacier = "hagenbrae"

#glacier="79Ncrac"
#glacier="ryder"
#glacier="ostenfeld"
#glacier="steensby"
#glacier="storstrommen"
#glacier="zachariae"
#glacier="upernavik"
glacier="upernavik_s"

#create output directory
os.makedirs(os.path.join(wd, glacier,'withlogo'), exist_ok = True)

In [7]:
logo_file = 'logoIGE.png'
logoIm = Image.open(logo_file)
logoWidth, logoHeight = logoIm.size

wd =os.getcwd()
ImgPath_in = os.path.join(wd, glacier)
files_in = [file for file in os.listdir(ImgPath_in) if os.path.isfile(os.path.join(ImgPath_in, file)) and os.path.splitext(file)[1]=='.png']

im = Image.open(os.path.join(wd, glacier,files_in[0]))
width, height = im.size
print(width)

scaleRatio= min(width*0.07/logoWidth,height*0.07/logoHeight)
logoWidth_resize = int(logoWidth*scaleRatio)
logoHeight_resize = int(logoHeight*scaleRatio)
logoIm = logoIm.resize((logoWidth_resize, logoHeight_resize))

print('Logo resize to:'+ str(logoWidth_resize)+','+str(logoHeight_resize))

borderLogo = int(logoWidth_resize/2)

for filename in os.listdir('./'+ glacier):
    if not (filename.endswith('.png') ) or filename == logo_file:
        continue
    im = Image.open(os.path.join(wd, glacier,filename))
    
    im.paste(logoIm, (width - logoWidth_resize-borderLogo, borderLogo),logoIm)
    im.save(os.path.join(wd, glacier,'withlogo', filename))

1693
Logo resize to:66,67


### Create the GIF File

Using Pillow, we obtain a GIf with the native resolution of images, but it changes the color of the logo. For GIF Format, PIL encodes the color in 8 bits and unfortunately the IGE logo colors are not well defined in 8 bits. Using .convert('RGB') alters the colors of the images.

In [8]:
# https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif

ImgPath_in = os.path.join(wd, glacier,'withlogo')
files_in = [file for file in sorted(os.listdir(ImgPath_in)) if os.path.isfile(os.path.join(ImgPath_in, file))and os.path.splitext(file)[1]=='.png']
fp_out = "./"+glacier+"/"+glacier+"_2017_2021.gif"

img, *imgs = [Image.open(os.path.join(wd, glacier,'withlogo',f)) for f in files_in]

#imgsRGB=[image.convert('P') for image in imgs]

img.save(fp=fp_out, format='GIF', append_images=imgs,
         save_all=True, duration=700, loop=0)

### create the GIF using FFMPEG:
 - create a 10MB file by reducing the resolution a little: `ffmpeg  -framerate 3 -i petermann%*.png -loop 0 petermann_2014_2021.gif`


`ffmpeg -i petermann%*.png -vf palettegen palette.png`
`ffmpeg -v warning -i petermann%*.png -i palette.png  -lavfi "paletteuse,setpts=6*PTS" -y petermann_2014_2021.gif`